In [1]:
import pandas as pd

import os
import sys
import wave
import copy
import math
import tensorflow as tf
from keras.models import load_model


from features import *
from helper import *
import pyaudio
import struct
import numpy as np
import matplotlib.pyplot as plt
import time
from tkinter import TclError

# use this backend to display in separate Tk window
%matplotlib tk

# constants

TIME_SEC = 2
RATE = 44100                 # samples per second
CHUNK = int(44100 * TIME_SEC)     # 1024 * 20 #int(44100 / TIME_SEC)             # samples per frame
FORMAT = pyaudio.paInt16     # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone


# p = pyaudio.PyAudio()
# for i in range(p.get_device_count()):
#     print(p.get_device_info_by_index(i))

Using TensorFlow backend.


In [2]:
def plot_line_graph(graph, data):
    graph.axhline(y=0.5, color='r', linestyle='--')
    graph.plot(data['ang'], label="Anger", color='r')
    graph.plot(data['exc'], label="Excited", color='y')
    graph.plot(data['neu'], label="Neutral", color='g')
    graph.plot(data['sad'], label="Sad", color='b')
    graph.legend(loc="upper left")

    return graph

def plot_bar_graph (graph, data):
    graph.axhline(y=0.5, color='r', linestyle='--')
    graph.bar(x=0, height=data[0], label="Anger", color='r')
    graph.bar(x=1, height=data[1], label="Excited", color='y')
    graph.bar(x=2, height=data[2], label="Neutral", color='g')
    graph.bar(x=3, height=data[3], label="Sad", color='b')
    graph.legend(loc="upper left")

    return graph

In [3]:
# create matplotlib figure and axes
#fig, ax = plt.subplots(1, figsize=(15, 7))

# pyaudio class instance
p = pyaudio.PyAudio()

print("starting stram decleration")

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK,
    input_device_index = 3
)

starting stram decleration


In [4]:
# set up model
model = load_model("Trained_models/2_layer_LSTM.pickle")
cols = ['ang', 'exc', 'neu', 'sad']
df_pred_wav = pd.DataFrame([np.zeros(4)], columns=cols)
data_prev = []
graph_window = 30

In [5]:
# variable for plotting
x = np.arange(0, 2 * CHUNK, 2)
fig , ax = plt.subplots(2)

# basic formatting for the axes
ax[0].set_title('Emotion Prediction')
ax[0].set_xlabel('Time')
ax[0].set_ylabel('Confidence')

ax[0]=  plot_line_graph(ax[0], df_pred_wav)
ax[1] = plot_bar_graph(ax[1], [0,0,0,0])

# show the plot
plt.show(block=False)

# for measuring frame rate
frame_count = 0
start_time = time.time()

In [6]:
print("starting loop")
while (True):

    # binary data
    data_new = stream.read(CHUNK)

    #convert data to integers, make np array, then offset it by 127
    data_new = struct.unpack(str(2 * CHUNK) + 'B', data_new)

    data_int = np.append(data_prev, data_new)
   #data_int = data_new

    # Generate features from data
    st_features = calculate_features(data_int, RATE, None)
    st_features, _ = pad_sequence_into_array(st_features, maxlen=100)

    # reshape input from (34, 100) to (1, 100, 34)
    st_features = np.array([st_features.T])

    # predict on model
    wav_test_results = model.predict(st_features)

    df_pred_wav = df_pred_wav.append({cols[0]:wav_test_results[0][0],
                        cols[1]:wav_test_results[0][1],
                        cols[2]:wav_test_results[0][2],
                        cols[3]:wav_test_results[0][3]
    }, ignore_index=True)

    df_pred_wav_view = df_pred_wav.tail(graph_window)
    df_pred_wav_view.reset_index(drop=True, inplace=True)
    ax[0].cla()
    ax[1].cla()

    ax[0] = plot_line_graph(ax[0], df_pred_wav_view)

    last_res = wav_test_results[0]
    ax[1] = plot_bar_graph (ax[1], last_res)

    # update figure canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1

    except TclError:

        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)

        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))

    data_prev = data_new

starting loop


KeyboardInterrupt: 

In [ ]:
# # stop stream (6)
# stream.stop_stream()
# stream.close()
# wf.close()
#
# # close PyAudio (7)
# p.terminate()